In [10]:
import torch
import numpy as np
from torchtext import data

SEED = 1452 # for reproducibility
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize= 'spacy')
LABEL = data.LabelField(dtype=torch.float)


In [14]:
# Load dataset
with open('datasets/winemag-data-130k-v2.csv') as f:
    lines = f.readlines()

print(len(lines))

129976


In [15]:
# Split in train and test
# @Pierre: en fait on utilisera plutot le split de torchtext (voir plus bas)

TEST_SET_SIZE = .3

indices = list(range(len(lines)))
np.random.seed(SEED)
np.random.shuffle(indices)
split_index = int(TEST_SET_SIZE*len(lines))
train_indices = indices[:split_index]
test_indices = indices[split_index:]
train_set = [lines[k] for k in train_indices]
test_set = [lines[k] for k in test_indices]

print(len(train_set))
print(len(test_set))

38992
90984


In [31]:
# preprocess json
# (torchtext needs a file with a new json record per row and not a proper json)
import json
with open('datasets/winemag-data-130k-v2.json') as f:
    jsonfile = json.loads(f.read())
res = ''
print(str(jsonfile[0]))
print(json.dumps(jsonfile[0]))
for record in jsonfile:
    res += json.dumps(record) + '\n'

print(res[:1000])


{'region_1': 'Etna', 'price': None, 'designation': 'Vulkà Bianco', 'taster_twitter_handle': '@kerinokeefe', 'title': 'Nicosia 2013 Vulkà Bianco  (Etna)', 'taster_name': 'Kerin O’Keefe', 'description': "Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.", 'winery': 'Nicosia', 'points': '87', 'variety': 'White Blend', 'province': 'Sicily & Sardinia', 'country': 'Italy', 'region_2': None}
{"region_1": "Etna", "price": null, "designation": "Vulk\u00e0 Bianco", "taster_twitter_handle": "@kerinokeefe", "title": "Nicosia 2013 Vulk\u00e0 Bianco  (Etna)", "taster_name": "Kerin O\u2019Keefe", "description": "Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.", "winery": "Nicosia", "points": "87", "variety": "White Blend", "province": "Sicily & Sardinia", "

In [32]:
# Write preprocessed json to new file
with open('testfile', 'w') as out:
    out.write(res)

In [33]:
# Setup a dataset from the preprocessed json
train_dataset = data.TabularDataset(
    path='testfile',
    format='json',
    fields={'description': ('description', data.Field(sequential=True)),
            'points': ('points', data.Field(sequential=False))}
)

In [39]:
import random
vars(train_dataset.examples[0])
train_data, valid_data = train_dataset.split(random_state = random.seed(SEED))
print(train_data)